In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# System imports
from os import path
import math

# Third-Party Imports
import cv2
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import griddata
import scipy.sparse.linalg

# modify to where you store your project data including utils
datadir = "/Users/jho/cs445/final_project/Single-Image-Deblurring/" 

# utilfn = datadir + "utils"
# !cp -r "$utilfn" .
datafn = datadir + "data/"
!cp -r "$datafn" .

# # can change this to your output directory of choice
# !mkdir "images"
# !mkdir "images/outputs"

# # import starter code
# import utils
# from utils.io import read_image, write_image, read_hdr_image, write_hdr_image
# from utils.display import display_images_linear_rescale, rescale_images_linear
# from utils.hdr_helpers import gsolve
# from utils.hdr_helpers import get_equirectangular_image
# from utils.bilateral_filter import bilateral_filter

In [ ]:
blur_img_name = 'data/brick.jpeg'
blur_img = cv2.cvtColor(cv2.imread(blur_img_name), cv2.COLOR_BGR2RGB).astype('double') / 255.0 
plt.imshow(blur_img)
plt.show()